<a href="https://colab.research.google.com/github/evanwmeeks/Springboard/blob/main/Capstone%203/Amazon_Reviews_for_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Package and file imports

In [21]:
!pip install tf-models-official 
!pip install -U tensorflow
!pip install -U tensorflow-text


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [22]:
from google.colab import drive
drive.mount('Drive')

Drive already mounted at Drive; to attempt to forcibly remount, call drive.mount("Drive", force_remount=True).


In [23]:
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd
import seaborn as sns
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import bz2
import csv
import re
import os
import gc
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import shutil

import tensorflow_hub as hub
import tensorflow as tf
import tensorflow_text as text
from official.nlp import optimization  # to create AdamW optimizer


tf.get_logger().setLevel('ERROR')

from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [24]:
#Wrapping entire file import process in an if-else statement to speed up processing if the cleaned files have already
# been exported

if not os.path.exists('/content/Drive/MyDrive/Colab Notebooks/cleaned_test_text.csv') and not os.path.exists('/content/Drive/MyDrive/Colab Notebooks/cleaned_train_text.csv'):

  #Reading in BZ2 Files
  train_file = bz2.BZ2File('/content/Drive/MyDrive/Colab Notebooks/train.ft.txt.bz2')
  test_file = bz2.BZ2File('/content/Drive/MyDrive/Colab Notebooks/test.ft.txt.bz2')

  #Splitting files into their individual lines
  train_file_lines = train_file.readlines()
  test_file_lines = test_file.readlines()

  #Decoding the lines into parseable data
  train_file_lines = [x.decode('utf-8') for x in train_file_lines]
  test_file_lines = [x.decode('utf-8') for x in test_file_lines]

  #Function to convert the individual lines into array objects. Note: 
  #Label 0 = negative, Label 1 = positive review. 

  def load_extract(file):
    texts, labels = [], []
    for line in file: 
      labels.append(int(line[9])-1)
      texts.append(line[10:].strip())
    return np.array(labels), texts

  #Calling the function on the data
  train_labels, train_text = load_extract(train_file_lines)
  test_labels, test_text = load_extract(test_file_lines)

  #Taking only 10% of datasets because sheer size keeps crashing the notebook.
  sub_train_text = train_text[:360000]
  sub_train_labels = train_labels[:360000]
  sub_test_text = test_text[:360000]
  sub_test_labels = test_labels[:360000]

  #Quick check that everything above worked smoothly.
  print(sub_train_text[0])
  print(sub_train_labels[0], "\n")
  print(sub_test_text[0])
  print(sub_test_labels[0])


else: 

  pass

Stuning even for the non-gamer: This sound track was beautiful! It paints the senery in your mind so well I would recomend it even to people who hate vid. game music! I have played the game Chrono Cross but out of all of the games I have ever played it has the best music! It backs away from crude keyboarding and takes a fresher step with grate guitars and soulful orchestras. It would impress anyone who cares to listen! ^_^
1 

Great CD: My lovely Pat has one of the GREAT voices of her generation. I have listened to this CD for YEARS and I still LOVE IT. When I'm in a good mood it makes me feel better. A bad mood just evaporates like sugar in the rain. This CD just oozes LIFE. Vocals are jusat STUUNNING and lyrics just kill. One of life's hidden gems. This is a desert isle CD in my book. Why she never made it big is just beyond me. Everytime I play this, no matter black, white, young, old, male, female EVERYBODY says one thing "Who was that singing ?"
1


In [25]:
#Function to clean the data of stop words, URLs/Hyperlinks, numbers, whitespace, etc. 
def clean_texts(texts):
    stwords = stopwords.words('english')
    temp_texts = []
    for i in range(len(texts)):
        text = re.sub('\d','0',texts[i])
        if 'www.' in text or 'http:' in text or 'https:' in text or '.com' in text: 
            text = re.sub(r"([^ ]+(?<=\.[a-z]{3}))", " ", text)
        text = re.sub('[^a-zA-Z]', ' ', text)
        text = text.lower()
        text = text.split()
        text = [word for word in text if not word in stwords]
        text = ' '.join(text)
        temp_texts.append(text)
    return temp_texts

Cleaning the datasets

In [26]:
#Custom CSV file open/reader
def open_csv(file):
  with open(file, mode='r') as csv_file:
    result = []
    for line in csv_file: 
      result.append(line)
    return result

In [27]:
#Creating the train/test text sets if they haven't already been exported

if os.path.exists('/content/Drive/MyDrive/Colab Notebooks/cleaned_train_text.csv'):

  train_texts = open_csv('/content/Drive/MyDrive/Colab Notebooks/cleaned_train_text.csv')
  sub_train_labels = open_csv('/content/Drive/MyDrive/Colab Notebooks/sub_train_labels.csv')

else: 
  
    train_texts = clean_texts(sub_train_text)
    np.savetxt('/content/Drive/MyDrive/Colab Notebooks/cleaned_train_text.csv', train_texts, fmt ='%s')
    np.savetxt('/content/Drive/MyDrive/Colab Notebooks/sub_train_labels.csv', sub_train_labels, fmt ='%s')


if os.path.exists('/content/Drive/MyDrive/Colab Notebooks/cleaned_test_text.csv'):

  test_texts = open_csv('/content/Drive/MyDrive/Colab Notebooks/cleaned_test_text.csv')
  sub_train_labels = open_csv('/content/Drive/MyDrive/Colab Notebooks/sub_test_labels.csv')

else: 

    test_texts = clean_texts(sub_test_text)
    np.savetxt('/content/Drive/MyDrive/Colab Notebooks/cleaned_test_text.csv', test_texts, fmt ='%s')
    np.savetxt('/content/Drive/MyDrive/Colab Notebooks/sub_test_labels.csv', sub_test_labels, fmt ='%s')


**Data Modeling using a Multinomial and Logitistic Regression**

In [28]:
from sklearn.feature_extraction.text import CountVectorizer

count_vect = CountVectorizer()
count_vect.fit(train_texts)

train_texts_vec = count_vect.transform(train_texts)

test_texts_vec = count_vect.transform(test_texts)


In [29]:
nb = MultinomialNB()
nb.fit(train_texts_vec, sub_train_labels)

MultinomialNB()

In [30]:
y_pred = nb.predict(test_texts_vec)

In [31]:
print('Accuracy:', accuracy_score(sub_test_labels, y_pred))

Accuracy: 0.8422444444444445


In [32]:
lr_model = LogisticRegression(n_jobs=-1, max_iter=150)
lr_model.fit(train_texts_vec, sub_train_labels)

LogisticRegression(max_iter=150, n_jobs=-1)

In [33]:
pred_lr = lr_model.predict(test_texts_vec)
print('Accuracy:', accuracy_score(sub_test_labels, pred_lr))

Accuracy: 0.8840305555555555


In [34]:
a = 1201
sample = test_texts[a]
print(sample)

sample_vec = count_vect.transform([sample])
pred = lr_model.predict(sample_vec)
print('\npredicted label:',pred[0])
print('actual label:', test_labels[a])

leak breastfeed poop back diapers leek back baby poops got waste time change shirt poop whatever else poop got annoying

predicted label: 0
actual label: 0


**Data Modeling using Tensorflow's BERT (Bidirectional Encoder Representations from Transformers)**

In [35]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_texts, sub_train_labels, shuffle = True)

In [36]:

#Reading csv file into TensorFlow dataset and printing first element for verification
train_ds = tf.data.TextLineDataset(["cleaned_train_text.csv"])
test_ds = tf.data.TextLineDataset(["cleaned_test_text.csv"])

In [37]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")

bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [38]:
def get_sentence_embedding(sentences):
    preprocessed_text = bert_preprocess(sentences)
    return bert_encoder(preprocessed_text)['pooled_output']




In [ ]:
x=get_sentence_embedding([train_texts])
print(x)

In [40]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [41]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [42]:
model.fit(np.array(X_train), y_train, epochs=3)

Epoch 1/3
8438/8438 [==============================] - 2604s 307ms/step - loss: 0.5195 - accuracy: 0.7453 - precision: 0.7507 - recall: 0.7432
Epoch 2/3
8438/8438 [==============================] - 2591s 307ms/step - loss: 0.4889 - accuracy: 0.7661 - precision: 0.7722 - recall: 0.7624
Epoch 3/3
8438/8438 [==============================] - 2592s 307ms/step - loss: 0.4859 - accuracy: 0.7676 - precision: 0.7740 - recall: 0.7635
